In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import json 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import random

In [2]:
# Define credentials
client_ID = 'YOUR_CLIENT_ID' 
client_SECRET = 'YOUR_CLIENT_SECRET'  
redirect_url = 'http://localhost/'
auth_url = 'https://accounts.spotify.com/api/token'

credentials = SpotifyClientCredentials(client_id=client_ID, client_secret=client_SECRET)

data = {
    'grant_type': 'client_credentials',
    'client_id': client_ID,
    'client_secret': client_SECRET,
}
auth_response = requests.post(auth_url, data=data)
access_token = auth_response.json().get('access_token')

# Set up the authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope='playlist-modify-private', client_id=client_ID, client_secret=client_SECRET, redirect_uri=redirect_url))

## Get Available Genres

In [3]:
# Set the Authorization header with the access token
headers = {
    "Authorization": "Bearer " + access_token
}

# Send the GET request
data = requests.get("https://api.spotify.com/v1/recommendations/available-genre-seeds", headers=headers).json()

# Extract the list of seed genres. Seed genre of your choice must be chosen from this list
available_genres = data['genres']
print('available genres: \n', available_genres)

available genres: 
 ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post

In [4]:
def get_playlist_tracks(playlist_id_list):
    '''
    This function takes a list of URLs of spotify playlists and 
    returns the track IDs of tracks in the playlist (s) as a list. 
    
    URLs to spotify playlists can be obtained as follows: 
    More options -> Share -> Copy link to playlist
    
    '''
    track_ids = []
    for playlist_id in playlist_id_list:
        offset = 0
        limit = 100
        total = limit  # Initialize total to enter the loop

        while offset < total:
            playlist_tracks = sp.playlist_tracks(playlist_id, offset=offset, limit=limit)
            track_ids.extend([track['track']['id'] for track in playlist_tracks['items']])
            offset += limit
            total = playlist_tracks['total']
    return track_ids

def name_to_rui(name_list):
    '''
    This function takes a list of names of the artists and
    returns their URIs as a list. 
    
    Make sure that the names you give are identical to the artist names 
    that appear on Spotiy
    '''
    artist_uri_list = []
    for artist in artist_list:
        search_result = sp.search(q='artist:' + artist, type='artist', limit=1)
        items = search_result['artists']['items']
        if len(items) > 0:
            uri = items[0]['uri'][15:]
            artist_uri_list.append(uri)
    return artist_uri_list

In [19]:
# Define settings and filters

# SETTINGS 
endpoint_url = "https://api.spotify.com/v1/recommendations?" 
token = access_token
user_id = client_ID

# OUR FILTERS
limit = 10 # adjust this value based how many songs you want in your playlist
market = "US" 
seed_genres = "r-n-b"

if seed_genres not in available_genres:
    print('=== WARNING! pick a valid genre from the following ===', '\n',available_genres)

# Define a list of seed artists and playlists 
artist_list = ['Dominic Fike', 'Omar Apollo', 'Daniel Caesar', 'SZA'] # Just an examle!
playlist_url = ['url1', 'url2']

seed_artists = name_to_rui(artist_list)  # Extract artists' uris
seed_tracks = get_playlist_tracks(playlist_url) 

# Shuffles the tracks amongest given playlists for better selection
seed_tracks_shuffled = random.sample(seed_tracks, len(seed_tracks)) 

## Get song recs!
### go through the songs & see if you like them. You can also run again to get another selection.

In [22]:
## === PERFORM THE QUERY === ##
uris = []
ids = []
emp_list = []
query = f'{endpoint_url}limit={limit}&market={market}&seed_genres={seed_genres}'

for artist_id in seed_artists:
    query += f'&seed_artists={artist_id}'

batch_size = 100  # Define the number of seed tracks per request

for i in range(0, len(seed_tracks_shuffled), batch_size):
    track_batch = seed_tracks_shuffled[i:i + batch_size]
    for track_id in track_batch:
        query += f'&seed_tracks={track_id}'

    response = requests.get(query, headers={"Content-Type": "application/json", "Authorization": f"Bearer {token}"})
    json_response = response.json()

    for i, j in enumerate(json_response['tracks']):
        emp_list.append(j)
        ids.append(j['id']) # j 
        print(f"{i+1}) \"{j['name']}\" by {j['artists'][0]['name']}")

    # Reset the query for the next batch
    query = f'{endpoint_url}limit={limit}&market={market}&seed_genres={seed_genres}'
    for artist_id in seed_artists:
        query += f'&seed_artists={artist_id}'

print('\n')
print('Total', len(ids) ,'tracks extracted!')
print('You could also run again to get a new set of selections')


1) "cindy" by Jeremy Zucker
2) "Down in History" by True Being
3) "Young & Alive" by Bazzi
4) "Young Harleezy" by Jack Harlow
5) "Me or Sum (feat. Future & Lil Baby)" by Nardo Wick
6) "Window Seat" by Erykah Badu
7) "4EVER" by Clairo
8) "Breakdown" by Ar'mon & Trey
9) "My Boo" by Usher
10) "SHADOW" by OGK John
1) "Daunt" by Jelani Aryeh
2) "Your Life Your Time" by Christian Leave
3) "So Into You" by Tamia
4) "Motocross" by Huron John
5) "Wanna Be" by The Internet
6) "Breeze" by Marius
7) "ARE YOU WITH THAT?" by Vince Staples
8) "FAMJAM4000" by Jordan Ward
9) "you always play" by scyq
10) "Golden" by Jill Scott
1) "Portland" by Strawberry Milk Cult
2) "Fight the Vegans" by WILLIS
3) "Color TV" by Deal Casino
4) "Hands Down" by The Greeting Committee
5) "HEADSTART" by Ritt Momney
6) "Vickie" by Peach Pit
7) "Song From Hell" by Peach Tree Rascals
8) "Your Life Your Time" by Christian Leave
9) "Sadie, KT-22" by Ashes To Amber
10) "Just a Boy" by The Backseat Lovers
1) "Beach Boy" by BENEE


## Create an empy playlist & add the tracks

In [24]:
# Set up the authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope='playlist-modify-private', client_id=client_ID, client_secret=client_SECRET, redirect_uri=redirect_url))

# Create an empty playlist
playlist_name ="Demo" # "Public Python generated R&B playlist"
playlist_description = "Seed artists: Omar Apollo, Daniel Casear, Dominic Fike, SZR; seed playlists: Alt r&b, 2022. Seed genre: r-n-b. Python generated R&B playlist!"
user_id = sp.me()['id']  # Get the current user's ID

playlist = sp.user_playlist_create(user_id, playlist_name, public=True)
sp.playlist_change_details(playlist['id'], description=playlist_description)

# Print the ID of the newly created playlist
print("Playlist created named:", playlist_name)# with ID:", playlist['id'])

# Add songs to the playlist
sp.playlist_add_items(playlist['id'], ids)
print('All the songs have been added!')

Playlist created named: Demo
All the songs have been added!
